In [1]:
import nbimporter
import numpy as np
import sys
import cv2
from timeit import default_timer as timer
from DataParserForCNN import read_full_df, read_class

Importing Jupyter notebook from DataParserForCNN.ipynb


Using TensorFlow backend.


In [14]:
class_array = read_class()
# base_path = 'C:/Users/IML/Desktop/QuickDrawDataset/'
base_path = '/Volumes/JS/QuickDraw/'

In [15]:
full_df, train_id, valid_id = read_full_df(base_path, class_array)

	 load df   :  339/340                   zigzag   7 min 21 sec

	 load valid_set split:  339/340                   zigzag  10 min 48 sec

In [78]:
def drawing_to_image(drawing, H, W):

    point=[]
    time =[]
    
#     coordinates = []
    
    for t,(x,y) in enumerate(drawing):
        point.append(np.array((x,y),np.float32).T)
        time.append(np.full(len(x),t))

    point = np.concatenate(point).astype(np.float32)
    time  = np.concatenate(time).astype(np.int32)

    image  = np.full((H,W,3),0,np.uint8)
    x_max = point[:,0].max()
    x_min = point[:,0].min()
    y_max = point[:,1].max()
    y_min = point[:,1].min()
    w = x_max-x_min
    h = y_max-y_min
    #print(w,h)

    s = max(w,h)
    norm_point = (point-[x_min,y_min])/s
    norm_point = (norm_point-[w/s*0.5,h/s*0.5])*max(W,H)*0.85
    norm_point = np.floor(norm_point + [W/2,H/2]).astype(np.int32)

    T = time.max()+1
    for t in range(T):
        p = norm_point[time==t]
        x,y = p.T
        image[y,x]=255
        N = len(p)
        for i in range(N-1):
            x0,y0 = p[i]
            x1,y1 = p[i+1]
            cv2.line(image,(x0,y0),(x1,y1),(255,255,255),1,cv2.LINE_AA)

#     return np.transpose(image, (2,0,1))
    return image


def null_augment(drawing, label, index):
#     cache = Struct(drawing = drawing.copy(), label = label, index=index)
    image = drawing_to_image(drawing, 64, 64)
    return image, label

In [2]:
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.parallel.data_parallel import data_parallel
from torch.utils.data.sampler import *
from model import shake_drop_net

In [80]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

In [81]:
class DoodleDataset(Dataset):

    def __init__(self, 
                 full_df, 
                 datasplit_id,
                 shuffle=False,
                 augment=null_augment, 
                 mode='simplified'):
        super(DoodleDataset, self).__init__()
        assert mode in ['simplified', 'raw']

        self.datasplit_id = datasplit_id
        self.augment = augment
        self.mode = mode

        self.df = full_df
        self.id = datasplit_id
        
         ### shuffle                    
        if shuffle:
            np.random.shuffle(self.datasplit_id)
        print('\n')

    def __str__(self):
        N = len(self.id)
        string = '' \
                 + '\tmode         = %s\n' % self.mode \
                 + '\tlen(self.id) = %d\n' % N \
                 + '\n'
        return string

    def __getitem__(self, index):

#         if self.mode == 'train':
        label, drawing_id, key_id = self.id[index]
        drawing = self.df[label]['drawing'][drawing_id]
        drawing = eval(drawing)

#         if self.mode == 'test':
#             label = None
#             drawing = self.df['drawing'][index]
#             drawing = eval(drawing)

        return self.augment(drawing, label, index)

    def __len__(self):
        return len(self.id)

In [82]:
def null_collate(batch):
    batch_size = len(batch)
#     cache = []
    input = []
    truth = []
    for b in range(batch_size):
        input.append(batch[b][0])
        truth.append(batch[b][1])
#         cache.append(batch[b][2])

    input = np.array(input).transpose(0, 3, 1, 2)
    input = torch.from_numpy(input).float()

    if truth[0] is not None:
        truth = np.array(truth)
        truth = torch.from_numpy(truth).long()

    return input, truth

In [83]:
train_dataset = DoodleDataset(full_df, train_id)

In [84]:
train_loader = DataLoader(train_dataset,
                          sampler=RandomSampler(train_dataset),
                          batch_size=128,
                          drop_last=True,
                          num_workers=2,
                          collate_fn=null_collate)

valid_dataset = DoodleDataset(full_df, valid_id, null_augment)

valid_loader = DataLoader(valid_dataset,
#                           sampler=RandomSampler(valid_dataset),
                          batch_size=128,
                          drop_last=False,
                          num_workers=2,
                          collate_fn=null_collate)

In [85]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(train_loader))

49680379
27200
388127


In [86]:
np.array(train_dataset[0][0]).shape

(64, 64, 3)

In [74]:
import os
from datetime import datetime
out_dir = './results/'

IDENTIFIER = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

log = Logger()
log.open(out_dir + '/log.train.txt', mode='a')
log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
log.write('\tout_dir      = %s\n' % out_dir)
log.write('\n')
log.write('\t<additional comments>\n')
log.write('\t  ... shakedrop on resnet50  ... \n')
log.write('\n')


--- [START 2018-11-03_15-57-19] ----------------------------------------------------------------

	out_dir      = ./results/

	<additional comments>
	  ... shakedrop on resnet50  ... 



In [75]:
log.write('** dataset setting **\n')
batch_size = 128
log.write('batch_size = %d\n' % (batch_size))
log.write('train_dataset : \n%s\n' % (train_dataset))
log.write('valid_dataset : \n%s\n' % (valid_dataset))
log.write('\n')

** dataset setting **
batch_size = 1024
train_dataset : 
	mode         = simplified
	len(self.id) = 49680379


valid_dataset : 
	mode         = simplified
	len(self.id) = 27200





In [3]:
# log.write('** net setting **\n')
net = shake_drop_net()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

if device == 'cuda':
    
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [4]:
net

ShakeDropNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (1): BasicBlock(
      (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(20, 23, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(23, eps=1e-05, momentum=0.1, affi

In [89]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-4,
                      momentum=0.9, weight_decay=1e-4)

In [10]:
def metric(logit, truth, is_average=True):

    with torch.no_grad():
        prob = F.softmax(logit, 1)
        value, top = prob.topk(3, dim=1, largest=True, sorted=True)
        correct = top.eq(truth.view(-1, 1).expand_as(top))

        if is_average==True:
            # top-3 accuracy
            correct = correct.float().sum(0, keepdim=False)
            correct = correct/len(truth)

            top = [correct[0], correct[0]+correct[1], correct[0]+correct[1]+correct[2]]
            precision = correct[0]/1 + correct[1]/2 + correct[2]/3
            return precision, top

        else:
            return correct

In [11]:
def train():
    net.train()
    train_loss = np.zeros(6, np.float32)
    valid_loss = np.zeros(6, np.float32)
    batch_loss = np.zeros(6, np.float32)
    num_iters = 300 * 1000
    iter = 0
    iter_save_interval = 2000
    iter_smooth = 20
    iter_valid = 100
    start_iter = 0
    iter_save = [0, num_iters - 1] + list(range(0, num_iters, iter_save_interval))  # 1*1000
    start_epoch = 0
    rate = 0
    i = 0

    start = timer()
    while iter < num_iters:
        sum_train_loss = np.zeros(6, np.float32)
        sum = 0

        optimizer.zero_grad()
        for inputs, truth in train_loader:

            len_train_dataset = len(train_dataset)
            batch_size = 1024
            iter = i + start_iter
            epoch = (iter - start_iter) * batch_size / len_train_dataset + start_epoch
            num_samples = epoch * len_train_dataset

#             if (iter % iter_valid == 0) and (iter != 0):
#                 net.set_mode('valid')
#                 valid_loss = do_valid(net, valid_loader, criterion)
#                 net.set_mode('train')

#                 asterisk = '*' if iter in iter_save else ' '
#                 ##--------

#                 print('\r', end='', flush=True)
#                 log.write('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % ( \
#                     rate, iter / 1000, epoch,
#                     valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3], asterisk,
#                     train_loss[0], train_loss[1], train_loss[2], train_loss[3],
#                     time_to_str((timer() - start), 'min'))
#                           )
#                 log.write('\n')
#                 time.sleep(0.01)

            if iter in iter_save:
                torch.save(net.state_dict(), out_dir + 'checkpoint/%08d_model.pth' % (iter))
                torch.save({
                    # 'optimizer': optimizer.state_dict(),
                    'iter': iter,
                    'epoch': epoch,
                }, out_dir + 'checkpoint/%08d_optimizer.pth' % (iter))

            # learning rate schduler -------------
#             lr = schduler(iter)
#             if lr < 0: 
#                 break
#             adjust_learning_rate(optimizer, lr)
#             rate = get_learning_rate(optimizer)

            # one iteration update  -------------
            # net.set_mode('train',is_freeze_bn=True)

#             input = input.cuda()
#             truth = truth.cuda()

            outputs = net(inputs)
            loss = criterion(output, truth)
            precision, top = metric(output, truth)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # print statistics  ------------
            batch_loss[:4] = np.array((loss.item(), top[0].item(), top[2].item(), precision.item(),))
            sum_train_loss += batch_loss
            sum += 1
            if iter % iter_smooth == 0:
                train_loss = sum_train_loss / sum
                sum_train_loss = np.zeros(6, np.float32)
                sum = 0

            print('\r', end='', flush=True)
            print('%0.4f %5.1f %6.1f | %0.3f  %0.3f  %0.3f  (%0.3f)%s  | %0.3f  %0.3f  %0.3f  (%0.3f)  | %s' % ( \
                rate, iter / 1000, epoch,
                valid_loss[0], valid_loss[1], valid_loss[2], valid_loss[3], ' ',
                batch_loss[0], batch_loss[1], batch_loss[2], batch_loss[3],
                time_to_str((timer() - start), 'min'))
                  , end='', flush=True)
            i = i + 1

    if 1:  # save last
        torch.save(net.state_dict(), out_dir + 'checkpoint/%d_model.pth' % (i))
        torch.save({
            'optimizer': optimizer.state_dict(),
            'iter': i,
            'epoch': epoch,
        }, out_dir + 'checkpoint/%d_optimizer.pth' % (i))

    log.write('\n')


In [12]:
train()

NameError: name 'net' is not defined

In [90]:
def run_train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        print(np.array(inputs).shape)
        
        outputs = net(inputs)
        print(outputs)
        
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 10 == 0:
            print('epoch : {} [{}/{}]| loss: {:.3f} | acc: {:.3f}'.format(epoch, batch_idx,
                                                                          len(train_loader),
                                                                          train_loss / (batch_idx + 1),
                                                                          100. * correct / total))

In [ ]:
run_train(20)

(128, 3, 64, 64)
